Author: Alexander Hoogsteyn \
Date: 26th September 2023

# Use of Proximal atomic Coordination in Energy system modelling

## 3. Introduction to Proxiamal Atomic Coordination (PAC)

For a detailed description of PAC, you are referred to: (Chapter 4 in particular) 

Romvary, Jordan (Jordan Joseph). 2018. “A Proximal Atomic Coordination Algorithm for Distributed Optimization in Distribution Grids.” Thesis, Massachusetts Institute of Technology. https://dspace.mit.edu/handle/1721.1/117843.
### 3.1 Example implementation

In [1]:
using JuMP
using Gurobi  # You can choose a different solver if you prefer

# Define the parameters of the problem
num_generators = 3
demand = 100  # Total electricity demand in MW

# Parameters for each generator
max_capacity = [50, 30, 40]  # Maximum power output for each generator in MW
marginal_cost = [20, 25, 30]  # Cost per unit power for each generator in EUR/MW

N = 1:num_generators
models = Dict()
λ = [0, 0, 0]
ρ = 1.0

λ_results =  [[0.0], [0.0], [0.0]]
g_results = [[0.0], [0.0], [0.0]]
p_results = [[0.0], [0.0], [0.0]]
g_bar = zeros(num_generators)
p_bar = zeros(num_generators)

function build_atom_1(N,GUROBI_ENV,marginal_cost,max_capacity)
    model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    
    g = model.ext[:variables][:g] = @variable(model, lower_bound=0, base_name="generation" )
    p = model.ext[:variables][:p] = @variable(model, lower_bound=0, base_name="line_power" )
    
    g_bar = 0
    p_bar = 0
    
    @objective(model, Min, marginal_cost*g + 1/(2*ρ)*(g-g_bar)^2 + 1/(2*ρ) * (p-p_bar)^2)
    
    @constraint(model, g<= max_capacity)
    
    @constraint(model, p == g)

    return model
end

function build_atom_2(N,GUROBI_ENV,marginal_cost,max_capacity,λ,p_1)
    model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    
    g = model.ext[:variables][:g] = @variable(model, lower_bound=0, base_name="generation" )
    p = model.ext[:variables][:p] = @variable(model, lower_bound=0, base_name="line_power" )
    
    g_bar = 0
    p_bar = 0
    
    @objective(model, Min, marginal_cost*g -λ*(p_1+g-p) + 1/(2*ρ)*(g-g_bar)^2 + 1/(2*ρ) * (p-p_bar)^2)
    
    @constraint(model, g<= max_capacity)
    
    @constraint(model, p == g)

    return model
end

function build_atom_3(N,GUROBI_ENV,marginal_cost,max_capacity,λ,p_2)
    model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    
    g = model.ext[:variables][:g] = @variable(model, lower_bound=0, base_name="generation" )
    p = model.ext[:variables][:p] = @variable(model, lower_bound=0, base_name="line_power" )
    
    g_bar = 0
    p_bar = 0
    
    @objective(model, Min, marginal_cost*g -λ*(p_2+g-p) + 1/(2*ρ)*(g-g_bar)^2 + 1/(2*ρ) * (p-p_bar)^2)

    @constraint(model, g<= max_capacity)
    
    @constraint(model, p == g)

    return model
end

function update_atom_1!(model,marginal_cost,g_bar,p_bar)
    g = model.ext[:variables][:g]
    p = model.ext[:variables][:p]
    
    @objective(model, Min, marginal_cost*g + 1/(2*ρ)*(g-g_bar)^2 + 1/(2*ρ) * (p-p_bar)^2)
    return model 
end

function update_atom_2!(model,marginal_cost,g_bar,p_bar,λ_2,p_1)
    g = model.ext[:variables][:g]
    p = model.ext[:variables][:p]
    
    @objective(model, Min, marginal_cost*g -λ_2*(p_1+g-p) + 1/(2*ρ)*(g-g_bar)^2 + 1/(2*ρ) * (p-p_bar)^2)
    return model 
end

function update_atom_3!(model,marginal_cost,g_bar,p_bar,λ_3,p_2)
    g = model.ext[:variables][:g]
    p = model.ext[:variables][:p]
    
    @objective(model, Min, marginal_cost*g -λ_3*(p_2+g-p) + 1/(2*ρ)*(g-g_bar)^2 + 1/(2*ρ) * (p-p_bar)^2)
    return model 
end


# Now we can put together the algorithm

models[1] = build_atom_1(N,GUROBI_ENV,marginal_cost[1],max_capacity[1])  
models[2] = build_atom_2(N,GUROBI_ENV,marginal_cost[2],max_capacity[2],0,0)
models[3] = build_atom_2(N,GUROBI_ENV,marginal_cost[3],max_capacity[3],0,0)

for i in 1:20

    # Optimize atoms
    for n in N
        optimize!(model[n])
    end

    # Broadcast solution to neighbouring atoms
    g_bar_1 = value(model[1].ext[:variables][:g])
    p_bar_1 = value(model[1].ext[:variables][:p])

    g_bar_2 = value(model[2].ext[:variables][:g])
    p_bar_2 = value(model[2].ext[:variables][:p])
    
    g_bar_3 = value(model[3].ext[:variables][:g])
    p_bar_3 = value(model[3].ext[:variables][:p])

    λ_2 = λ_2 + ρ * (p_bar_1+g_bar_2-p_bar_2)

    λ_3 = λ_3 + ρ * (p_bar_2+g_bar_3-p_bar_3)

    update_atom_1!(model[1],marginal_cost[1],g_bar_1,p_bar_1)
    update_atom_2!(model[2],marginal_cost[2],g_bar_2,p_bar_2,λ_2,p_bar_1)
    update_atom_3!(model[3],marginal_cost[3],g_bar_3,p_bar_3,λ_3,p_bar_2)

end

LoadError: UndefVarError: GUROBI_ENV not defined

This implementation is argueable overcomplicated for this simple model, but its point is to illustrate how the algorithm exploits structure to keep the optimization problem in each node relatively simple.

### 3.2 Improving PAC by adding momentum term (NST-PAC)

## 3.3 A more realistic application: solving an OPF with PAC

## References
Detailed description of PAC:

- Romvary, Jordan (Jordan Joseph). 2018. “A Proximal Atomic Coordination Algorithm for Distributed Optimization in Distribution Grids.” Thesis, Massachusetts Institute of Technology. https://dspace.mit.edu/handle/1721.1/117843.
